In [ ]:
# %% [markdown]
# # AI Personal Trainer — Demo notebook (Ollama + MediaPipe)
# Requisiti: Python 3.10+, Ollama in esecuzione su localhost:11434, webcam opzionale.
# Questo notebook supporta anche un file video o una simulazione senza camera.

In [12]:
# --- Cell 1: Config & imports ---
import os, time, json, math, re
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Tuple, Any

import cv2
import numpy as np
import requests
import mediapipe as mp

# Ollama
OLLAMA_URL = os.environ.get("OLLAMA_URL", "http://localhost:11434/api/generate")
OLLAMA_MODEL = os.environ.get("OLLAMA_MODEL", "llama3.1")

DISPLAY_SCALE = 0.9

user_profile = {
    "name": "Alex",
    "age": 32,
    "sex": "M",
    "height_cm": 178,
    "weight_kg": 76,
    "level": "intermediate",
    "goals": ["ricomposizione corporea", "forza", "resistenza"],
    "preferences": {
        "tone": "motivational_friendly",
        "cue_style": "visual_plus_short_verbal",
        "language": "it-IT"
    },
    "equipment": {"dumbbells": True, "mini_barbell": True, "bands": True, "mat": True},
    "constraints": {"lower_back_sensitivity": False, "shoulder_limitations": False, "knee_limitations": False}
}


In [22]:
# --- Cell 2: Pose + feature utils (riuso delle tue funzioni) ---
class PoseEstimator:
    def __init__(self, static_image_mode=False, model_complexity=1, enable_segmentation=False):
        self.mp_pose = mp.solutions.pose
        self.pose = self.mp_pose.Pose(
            static_image_mode=static_image_mode,
            model_complexity=model_complexity,
            enable_segmentation=enable_segmentation,
            smooth_landmarks=True,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )
        self.drawing = mp.solutions.drawing_utils
        self.drawing_styles = mp.solutions.drawing_styles

    def process(self, frame_bgr):
        frame_rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
        return self.pose.process(frame_rgb)

    def draw_landmarks(self, frame_bgr, landmarks, visibility_th=0.5, highlight: Dict[int, Tuple[int,int,int]]=None):
        if landmarks is None:
            return frame_bgr
        image = frame_bgr.copy()
        self.drawing.draw_landmarks(
            image, landmarks, self.mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=self.drawing_styles.get_default_pose_landmarks_style()
        )
        if highlight:
            h, w = image.shape[:2]
            for idx, color in highlight.items():
                lmk = landmarks.landmark[idx]
                if lmk.visibility >= visibility_th:
                    cx, cy = int(lmk.x*w), int(lmk.y*h)
                    cv2.circle(image, (cx, cy), 8, color, -1)
        return image

POSE = mp.solutions.pose.PoseLandmark

def _get_xy(landmarks, idx, frame_shape):
    h, w = frame_shape[:2]
    l = landmarks.landmark[idx]
    return np.array([l.x*w, l.y*h], dtype=np.float32), float(l.visibility)

def angle(a, b, c):
    ab = a - b; cb = c - b
    denom = (np.linalg.norm(ab)*np.linalg.norm(cb) + 1e-6)
    cosang = np.clip(np.dot(ab, cb)/denom, -1.0, 1.0)
    return math.degrees(math.acos(cosang))

def knee_angle(landmarks, side:str, frame_shape):
    hip,knee,ankle = (POSE.LEFT_HIP,POSE.LEFT_KNEE,POSE.LEFT_ANKLE) if side=="left" else (POSE.RIGHT_HIP,POSE.RIGHT_KNEE,POSE.RIGHT_ANKLE)
    A,_=_get_xy(landmarks, hip, frame_shape); B,_=_get_xy(landmarks, knee, frame_shape); C,_=_get_xy(landmarks, ankle, frame_shape)
    return angle(A,B,C)

def hip_angle(landmarks, side:str, frame_shape):
    shoulder,hip,knee = (POSE.LEFT_SHOULDER,POSE.LEFT_HIP,POSE.LEFT_KNEE) if side=="left" else (POSE.RIGHT_SHOULDER,POSE.RIGHT_HIP,POSE.RIGHT_KNEE)
    A,_=_get_xy(landmarks, shoulder, frame_shape); B,_=_get_xy(landmarks, hip, frame_shape); C,_=_get_xy(landmarks, knee, frame_shape)
    return angle(A,B,C)

def elbow_angle(landmarks, side:str, frame_shape):
    shoulder,elbow,wrist = (POSE.LEFT_SHOULDER,POSE.LEFT_ELBOW,POSE.LEFT_WRIST) if side=="left" else (POSE.RIGHT_SHOULDER,POSE.RIGHT_ELBOW,POSE.RIGHT_WRIST)
    A,_=_get_xy(landmarks, shoulder, frame_shape); B,_=_get_xy(landmarks, elbow, frame_shape); C,_=_get_xy(landmarks, wrist, frame_shape)
    return angle(A,B,C)

def line_point_distance(p1, p2, p):  # distanza del punto p dalla retta p1-p2
    # p1, p2, p sono np.array([x,y])
    num = np.abs(np.cross(p2 - p1, p1 - p))
    den = (np.linalg.norm(p2 - p1) + 1e-6)
    return float(num / den)

def pelvis_drop_norm(landmarks, frame_shape):
    Lhip,_ = _get_xy(landmarks, POSE.LEFT_HIP, frame_shape)
    Rhip,_ = _get_xy(landmarks, POSE.RIGHT_HIP, frame_shape)
    hip_mid = (Lhip + Rhip) / 2
    # normalizza su altezza "corpo" proxy (spalla->caviglia)
    Lsh,_ = _get_xy(landmarks, POSE.LEFT_SHOULDER, frame_shape)
    Lank,_ = _get_xy(landmarks, POSE.LEFT_ANKLE, frame_shape)
    body_len = np.linalg.norm(Lsh - Lank) + 1e-6
    return float((Lhip[1] - Rhip[1]) / body_len)  # positivo se anca sinistra "più bassa"

def compute_features(landmarks, frame_shape) -> Dict[str,float]:
    # base init per None
    if landmarks is None:
        keys = [
            "knee_ang","hip_ang","elbow_ang","shoulder_ang","ankle_ang",
            "depth","valgus_flag","knee_valgus_score",
            "left_knee_over_ankle","right_knee_over_ankle",
            "torso_tilt","pelvis_drop_norm","shoulder_hip_ankle_collinearity",
            "arm_symmetry","leg_symmetry","stance_width","grip_width",
            "bar_vertical_disp","shoulderY","wristY",
            "knee_to_chest_norm_L","knee_to_chest_norm_R",
            "crossbody_knee_wrist_prox_L","crossbody_knee_wrist_prox_R",
            "hands_to_feet_min_dist","tuck_compactness","arms_overhead"
        ]
        return {k:0.0 for k in keys}

    h, w = frame_shape[:2]

    # --- ANGOLI MEDI (già noti + nuovi) ---
    knee = (knee_angle(landmarks,"left",frame_shape)+knee_angle(landmarks,"right",frame_shape))/2
    hipA = (hip_angle(landmarks,"left",frame_shape)+hip_angle(landmarks,"right",frame_shape))/2
    elbow = (elbow_angle(landmarks,"left",frame_shape)+elbow_angle(landmarks,"right",frame_shape))/2
    # shoulder/ankle aggiuntivi
    def shoulder_angle(side):  # busto–spalla–gomito
        hip, shoulder, elbow_j = ((POSE.LEFT_HIP, POSE.LEFT_SHOULDER, POSE.LEFT_ELBOW) if side=="left"
                                  else (POSE.RIGHT_HIP, POSE.RIGHT_SHOULDER, POSE.RIGHT_ELBOW))
        A,_ = _get_xy(landmarks, hip, frame_shape)
        B,_ = _get_xy(landmarks, shoulder, frame_shape)
        C,_ = _get_xy(landmarks, elbow_j, frame_shape)
        return angle(A,B,C)
    def ankle_angle(side):     # ginocchio–caviglia–punta
        knee_j, ankle_j, foot_j = ((POSE.LEFT_KNEE, POSE.LEFT_ANKLE, POSE.LEFT_FOOT_INDEX) if side=="left"
                                   else (POSE.RIGHT_KNEE, POSE.RIGHT_ANKLE, POSE.RIGHT_FOOT_INDEX))
        A,_ = _get_xy(landmarks, knee_j, frame_shape)
        B,_ = _get_xy(landmarks, ankle_j, frame_shape)
        C,_ = _get_xy(landmarks, foot_j, frame_shape)
        return angle(A,B,C)
    shoulderA = (shoulder_angle("left")+shoulder_angle("right"))/2
    ankleA = (ankle_angle("left")+ankle_angle("right"))/2

    # --- PUNTI CHIAVE ---
    Lhip,_ = _get_xy(landmarks, POSE.LEFT_HIP, frame_shape)
    Rhip,_ = _get_xy(landmarks, POSE.RIGHT_HIP, frame_shape)
    Lkne,_ = _get_xy(landmarks, POSE.LEFT_KNEE, frame_shape)
    Rkne,_ = _get_xy(landmarks, POSE.RIGHT_KNEE, frame_shape)
    Lank,_ = _get_xy(landmarks, POSE.LEFT_ANKLE, frame_shape)
    Rank,_ = _get_xy(landmarks, POSE.RIGHT_ANKLE, frame_shape)
    Lwri,_ = _get_xy(landmarks, POSE.LEFT_WRIST, frame_shape)
    Rwri,_ = _get_xy(landmarks, POSE.RIGHT_WRIST, frame_shape)
    Lsho,_ = _get_xy(landmarks, POSE.LEFT_SHOULDER, frame_shape)
    Rsho,_ = _get_xy(landmarks, POSE.RIGHT_SHOULDER, frame_shape)

    mid_sho = (Lsho + Rsho)/2
    mid_hip = (Lhip + Rhip)/2
    mid_ank = (Lank + Rank)/2

    # --- PROFONDITÀ SQUAT ---
    depth = float( ((Lhip[1]+Rhip[1])/2) - ((Lkne[1]+Rkne[1])/2) )

    # --- VALGISMO NORMALIZZATO ---
    hip_width = abs(Rhip[0] - Lhip[0]) + 1e-6
    left_kot  = (Lkne[0] - Lank[0]) / hip_width
    right_kot = (Rkne[0] - Rank[0]) / hip_width
    knee_valgus_score = max(0.0, -left_kot) + max(0.0, right_kot)
    valgus_flag = 1 if knee_valgus_score > 0.08 else 0  # soglia di esempio

    # --- TORSO / COLLINEARITÀ PER PLANK ---
    # torso_tilt: angolo rispetto verticale del vettore hip->shoulder (media L/R)
    torso_vec = mid_sho - mid_hip
    torso_tilt = math.degrees(math.atan2(torso_vec[0], torso_vec[1]))  # 0 = verticale

    # distanza (normalizzata) dell'anca dalla retta spalla–caviglia (collinearity error)
    sha = mid_sho; ank = mid_ank
    hip_line_dist = line_point_distance(sha, ank, mid_hip)
    norm_len = (np.linalg.norm(sha - ank) + 1e-6)
    shoulder_hip_ankle_collinearity = float(hip_line_dist / norm_len)

    # --- SIMMETRIE & LARGHEZZE ---
    arm_symmetry = float(elbow_angle(landmarks,"left",frame_shape) - elbow_angle(landmarks,"right",frame_shape))
    leg_symmetry = float(knee_angle(landmarks,"left",frame_shape)  - knee_angle(landmarks,"right",frame_shape))
    stance_width = float(abs(Lank[0] - Rank[0]))
    grip_width   = float(abs(Lwri[0] - Rwri[0]))

    # --- ROM verticale (press/trazioni) ---
    shoulderY = float((Lsho[1] + Rsho[1]) / 2)
    wristY    = float((Lwri[1] + Rwri[1]) / 2)
    bar_vertical_disp = float(shoulderY - wristY)

    # --- MOUNTAIN CLIMBER: knee->chest & cross-body ---
    # normalizzazione su lunghezza gamba (anca->caviglia)
    leg_len = float(np.linalg.norm(mid_hip - mid_ank) + 1e-6)
    knee_to_chest_norm_L = float(np.linalg.norm(Lkne - mid_sho) / leg_len)
    knee_to_chest_norm_R = float(np.linalg.norm(Rkne - mid_sho) / leg_len)
    # prossimità ginocchio-pols(o) opposto (più piccolo = più "cross")
    crossbody_knee_wrist_prox_L = float(np.linalg.norm(Lkne - Rwri) / (norm_len + 1e-6))
    crossbody_knee_wrist_prox_R = float(np.linalg.norm(Rkne - Lwri) / (norm_len + 1e-6))

    # --- BURPEES: mani <-> piedi & tuck compactness & braccia sopra la testa ---
    hands_to_feet_min_dist = float(min(
        np.linalg.norm(Lwri - Lank), np.linalg.norm(Lwri - Rank),
        np.linalg.norm(Rwri - Lank), np.linalg.norm(Rwri - Rank)
    ) / (norm_len + 1e-6))
    # tuck: ginocchia molto vicine al torace (valore piccolo = ottimo tuck)
    tuck_compactness = float(min(knee_to_chest_norm_L, knee_to_chest_norm_R))
    # braccia sopra la testa: polsi più in alto (y minore) delle spalle
    arms_overhead = 1.0 if wristY < shoulderY - 10 else 0.0  # 10 px come margine

    return {
        "knee_ang":float(knee),"hip_ang":float(hipA),"elbow_ang":float(elbow),
        "shoulder_ang":float(shoulderA),"ankle_ang":float(ankleA),
        "depth":depth,
        "valgus_flag":int(valgus_flag),
        "knee_valgus_score":float(knee_valgus_score),
        "left_knee_over_ankle":float(left_kot),
        "right_knee_over_ankle":float(right_kot),
        "torso_tilt":float(torso_tilt),
        "pelvis_drop_norm":float(pelvis_drop_norm(landmarks, frame_shape)),
        "shoulder_hip_ankle_collinearity":float(shoulder_hip_ankle_collinearity),
        "arm_symmetry":arm_symmetry,"leg_symmetry":leg_symmetry,
        "stance_width":stance_width,"grip_width":grip_width,
        "bar_vertical_disp":bar_vertical_disp,
        "shoulderY":shoulderY,"wristY":wristY,
        "knee_to_chest_norm_L":knee_to_chest_norm_L,
        "knee_to_chest_norm_R":knee_to_chest_norm_R,
        "crossbody_knee_wrist_prox_L":crossbody_knee_wrist_prox_L,
        "crossbody_knee_wrist_prox_R":crossbody_knee_wrist_prox_R,
        "hands_to_feet_min_dist":hands_to_feet_min_dist,
        "tuck_compactness":tuck_compactness,
        "arms_overhead":arms_overhead
    }

In [23]:
import cv2, numpy as np, os, requests

def safe_imread(path_or_url:str, fallback_size=(480,640)):
    """
    - Se path_or_url è un file esistente -> lo carica.
    - Altrimenti tenta di scaricarlo come URL e salvarlo in 'test_input.jpg'.
    - Se fallisce, ritorna un frame nero (placeholder).
    """
    img = None
    if os.path.exists(path_or_url):
        img = cv2.imread(path_or_url)
    else:
        try:
            r = requests.get(path_or_url, timeout=10)
            r.raise_for_status()
            with open("test_input.jpg", "wb") as f:
                f.write(r.content)
            img = cv2.imread("test_input.jpg")
        except Exception:
            img = None

    if img is None:
        img = np.zeros((fallback_size[0], fallback_size[1], 3), dtype=np.uint8)
        print("⚠️ Immagine non trovata/non scaricata: uso placeholder nero.")
    return img

In [24]:
# Opzione A: file locale
img = safe_imread("../data/image/squat.jpg")

# Opzione B: URL (incolla un link a una foto intera di una persona)
# img = safe_imread("https://.../una_foto_intera.jpg")

In [25]:
# Richiede: PoseEstimator definita nelle celle precedenti
pe = PoseEstimator(static_image_mode=True)

frame = img  # usa l'immagine caricata con safe_imread
res = pe.process(frame)

if res and res.pose_landmarks:
    out = pe.draw_landmarks(frame, res.pose_landmarks)
    cv2.imwrite("keypoints_preview.jpg", out)
    print("✅ Keypoints disegnati. File: keypoints_preview.jpg")
    print("Landmarks rilevati:", len(res.pose_landmarks.landmark))  # atteso 33
else:
    print("❌ Nessun corpo rilevato: prova un'altra foto (intera, ben illuminata).")


✅ Keypoints disegnati. File: keypoints_preview.jpg
Landmarks rilevati: 33


I0000 00:00:1757337743.582121 6326649 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4 Max
W0000 00:00:1757337743.630497 6605810 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1757337743.637897 6605823 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [26]:
# Richiede: compute_features(...) definita nelle celle precedenti
if res and res.pose_landmarks:
    feats = compute_features(res.pose_landmarks, frame.shape)
    # stampa ordinata e leggibile
    nice = {k: (round(v,2) if isinstance(v, float) else v) for k,v in feats.items()}
    print("📊 Features:", nice)
else:
    print("❌ Niente features: nessun landmark trovato.")

📊 Features: {'knee_ang': 154.94, 'hip_ang': 147.23, 'elbow_ang': 111.72, 'shoulder_ang': 68.83, 'ankle_ang': 164.2, 'depth': -29.81, 'valgus_flag': 1, 'knee_valgus_score': 0.19, 'left_knee_over_ankle': -0.19, 'right_knee_over_ankle': -0.13, 'torso_tilt': -177.26, 'pelvis_drop_norm': -0.02, 'shoulder_hip_ankle_collinearity': 0.02, 'arm_symmetry': -72.29, 'leg_symmetry': 40.81, 'stance_width': 75.45, 'grip_width': 143.59, 'bar_vertical_disp': 0.64, 'shoulderY': 150.37, 'wristY': 149.73, 'knee_to_chest_norm_L': 1.13, 'knee_to_chest_norm_R': 1.17, 'crossbody_knee_wrist_prox_L': 0.91, 'crossbody_knee_wrist_prox_R': 0.73, 'hands_to_feet_min_dist': 1.0, 'tuck_compactness': 1.13, 'arms_overhead': 0.0}


In [27]:
# Dizionario: nome feature -> descrizione da passare al LLM
FEATURE_DOCS = {
  "knee_ang": "Gradi al ginocchio (anca-ginocchio-caviglia). 180=esteso, ~60-100 in accosciata.",
  "hip_ang": "Gradi all’anca (spalla-anca-ginocchio). Più basso = più flessione d’anca.",
  "elbow_ang": "Gradi al gomito (spalla-gomito-polso). 180=braccio esteso.",
  "shoulder_ang": "Gradi alla spalla (anca-spalla-gomito). Usa per press/alzate.",
  "ankle_ang": "Gradi alla caviglia (ginocchio-caviglia-punta). Usa per squat/affondi.",
  "depth": "Differenza verticale anca-ginocchio (px). Negativo = anca più bassa del ginocchio.",
  "valgus_flag": "1 se le ginocchia collassano verso il centro (proxy 2D normalizzata), altrimenti 0.",
  "knee_valgus_score": "Entità del collasso mediale ginocchia (normalizzato su larghezza bacino).",
  "left_knee_over_ankle": "Offset orizzontale ginocchio-sx vs caviglia-sx (normalizzato bacino). Negativo = verso il centro.",
  "right_knee_over_ankle": "Offset orizzontale ginocchio-dx vs caviglia-dx (normalizzato bacino). Positivo = verso il centro.",
  "torso_tilt": "Inclinazione busto rispetto alla verticale (°). 0≈verticale; >0=più inclinato.",
  "pelvis_drop_norm": "Differenza altezza anche L-R normalizzata (asimmetria bacino).",
  "shoulder_hip_ankle_collinearity": "Distanza normalizzata dell’anca dalla linea spalla-caviglia (plank line error).",
  "arm_symmetry": "Differenza gradi gomito L-R (simmetria braccia).",
  "leg_symmetry": "Differenza gradi ginocchia L-R (simmetria gambe).",
  "stance_width": "Distanza orizzontale tra caviglie (px).",
  "grip_width": "Distanza orizzontale tra polsi (px).",
  "bar_vertical_disp": "ROM verticale mani vs spalle (px). >0 in press; <0 in trazione.",
  "shoulderY": "Quota verticale media spalle (px, origine in alto).",
  "wristY": "Quota verticale media polsi (px, origine in alto).",
  "knee_to_chest_norm_L": "Distanza ginocchio-sx → torace (spalle) normalizzata (MC): più piccola = più vicino.",
  "knee_to_chest_norm_R": "Distanza ginocchio-dx → torace normalizzata.",
  "crossbody_knee_wrist_prox_L": "Distanza ginocchio-sx ↔ polso-dx normalizzata (MC cross).",
  "crossbody_knee_wrist_prox_R": "Distanza ginocchio-dx ↔ polso-sx normalizzata.",
  "hands_to_feet_min_dist": "Distanza minima mano-piede normalizzata (burpee tuck/raccolta).",
  "tuck_compactness": "Compattamento tuck (min distanza ginocchio→torace normalizzata).",
  "arms_overhead": "1 se i polsi stanno sopra le spalle (salto mani in alto), altrimenti 0."
}

In [31]:
import json, requests, os

# Se non l'hai già definito:
OLLAMA_URL = os.environ.get("OLLAMA_URL", "http://localhost:11434/api/generate")
OLLAMA_MODEL = os.environ.get("OLLAMA_MODEL", "llama3.1")

LLM_JUDGE_SYSTEM = """Sei un coach di fitness AI. Valuti un singolo frame di un esercizio usando SOLO le feature numeriche fornite.
NON inventare ciò che non è nelle feature. Rispondi SOLO JSON con lo schema:
{
  "verdict": "correct" | "minor_issue" | "major_issue",
  "issue_tags": string[],        // es. ["depth_short","valgus"]
  "cues": string[],              // max 3 correzioni brevi, in italiano
  "confidence": number,          // 0..1
  "notes": string                // opzionale (breve)
}
Linee guida:
- Se le misure sono incomplete/ambigue, usa 'minor_issue' e spiega brevemente in 'notes'.
- Le 'cues' devono essere concrete e azionabili (es. “Spingi le ginocchia verso l’esterno”).
"""

def build_llm_judge_prompt(exercise_name:str, camera_view:str, profile:dict,
                           features:dict, feature_docs:dict, mode:str="reps") -> str:
    payload = {
        "exercise": exercise_name,
        "mode": mode,                      # "reps" | "time"
        "camera_view": camera_view,        # "frontal" | "lateral" | "45deg"
        "user_level": profile.get("level","intermediate"),
        "goals": profile.get("goals", []),
        "tone": profile.get("preferences",{}).get("tone","motivational_friendly"),
        "feature_docs": feature_docs,      # definizioni (glossario)
        "features": features               # valori istantanei
    }
    return f"{LLM_JUDGE_SYSTEM}\nDati:\n{json.dumps(payload, ensure_ascii=False)}\nRispondi SOLO JSON."

def ollama_json(prompt:str, model:str=OLLAMA_MODEL, temperature:float=0.2, timeout:int=20) -> dict:
    try:
        r = requests.post(OLLAMA_URL, json={
            "model": model, "prompt": prompt, "stream": False, "options":{"temperature": temperature}
        }, timeout=timeout)
        r.raise_for_status()
        txt = r.json().get("response","").strip()
        return json.loads(txt)
    except Exception as e:
        return {"_error": str(e)}


In [33]:
def llm_judge_one_frame(image_path:str, exercise_name:str, camera_view:str="frontal",
                        profile:dict=None, draw_overlay:bool=True):
    import cv2, numpy as np

    if profile is None:
        profile = {
            "level": "intermediate",
            "goals": ["ricomposizione corporea"],
            "preferences": {"tone":"motivational_friendly"}
        }

    # 1) carica immagine
    img = cv2.imread(image_path)
    assert img is not None, f"Immagine non trovata: {image_path}"

    # 2) keypoints & features
    pe = PoseEstimator(static_image_mode=True)
    res = pe.process(img)
    if not (res and res.pose_landmarks):
        print("❌ Nessun corpo rilevato. Prova con un’immagine a figura intera e ben illuminata.")
        return None

    feats = compute_features(res.pose_landmarks, img.shape)

    # 3) prepara prompt e chiama LLM
    prompt = build_llm_judge_prompt(exercise_name, camera_view, profile, feats, FEATURE_DOCS, mode="reps")
    judge = ollama_json(prompt)

    # 4) overlay semplice
    out = pe.draw_landmarks(img, res.pose_landmarks)
    y = 28
    header = f"Exercise: {exercise_name} | View: {camera_view}"
    cv2.putText(out, header, (20,y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (20,220,20), 2); y += 30

    if isinstance(judge, dict) and "_error" not in judge:
        verdict = judge.get("verdict","?")
        conf = judge.get("confidence", None)
        cues = judge.get("cues", [])[:3]
        issues = judge.get("issue_tags", [])[:3]
        line1 = f"Verdict: {verdict}" + (f"  (conf {conf:.2f})" if isinstance(conf,(int,float)) else "")
        cv2.putText(out, line1, (20,y), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255,180,30), 2); y += 26
        if issues:
            cv2.putText(out, "Issues: " + ", ".join(issues), (20,y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (50,200,255), 2); y += 24
        for c in cues:
            cv2.putText(out, f"• {c}", (20,y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (50,200,255), 2); y += 24
    else:
        cv2.putText(out, f"LLM error: {judge.get('_error','?')}", (20,y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)

    # 5) salva risultato
    out_path = "llm_judge_overlay.jpg"
    cv2.imwrite(out_path, out)
    print("✅ Giudizio LLM:", json.dumps(judge, ensure_ascii=False))
    print("Overlay salvato in:", out_path)
    return judge


In [34]:
# Esempi (usa un’immagine coerente col gesto):
llm_judge_one_frame("../data/image/squat.jpg", exercise_name="goblet_squat", camera_view="frontal")
# llm_judge_one_frame("frame_pushup.jpg", exercise_name="push_up", camera_view="lateral")
# llm_judge_one_frame("frame_plank.jpg", exercise_name="plank", camera_view="lateral")


I0000 00:00:1757340068.707136 6326649 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M4 Max
W0000 00:00:1757340068.754198 6636465 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1757340068.761636 6636465 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


✅ Giudizio LLM: {"verdict": "minor_issue", "issue_tags": ["valgus_flag"], "cues": ["Assicurati che le ginocchia non collassino verso il centro", "Spingi le ginocchie verso l'esterno"], "confidence": 0.8, "notes": "Misura ambigua per la valgus flag, ma sembra che ci sia un leggero collasso delle ginocchia"}
Overlay salvato in: llm_judge_overlay.jpg


{'verdict': 'minor_issue',
 'issue_tags': ['valgus_flag'],
 'cues': ['Assicurati che le ginocchia non collassino verso il centro',
  "Spingi le ginocchie verso l'esterno"],
 'confidence': 0.8,
 'notes': 'Misura ambigua per la valgus flag, ma sembra che ci sia un leggero collasso delle ginocchia'}

In [ ]:
# --- Cell 5: prompts ---

SYSTEM_PLAN = """Sei un personal trainer AI. Parla e pensa in italiano.
Riceverai un profilo utente e l'attrezzatura disponibile. Genera un PIANO SEGRETO di allenamento adatto a livello e obiettivi.
NON rivelare l'intero piano all'utente: fornisci solo JSON col piano interno.
Il piano deve includere per ciascun esercizio i TARGET TECNICI (soglie angoli/criteri).
Rispondi SOLO JSON con lo schema:

{
  "plan_id": "string",
  "blocks": [
    {
      "id": "B1",
      "type": "warmup|strength|conditioning|finisher",
      "duration_s": 300,
      "sequence": [
        {
          "exercise": "goblet_squat",
          "equipment": ["dumbbells"],
          "mode": "reps|time",
          "target": {"reps": 12, "sets": 3, "tempo": "3010", "rest_s": 60} or {"time_s": 40, "rounds": 6, "rest_s": 20},
          "technique": {
            "vars": {"knee_top":160, "knee_bottom":100},   // soglie personalizzate
            "transitions": [
              {"from":"top","to":"down","when_all":["knee_ang < knee_top"]},
              {"from":"down","to":"bottom","when_all":["knee_ang <= knee_bottom"]},
              {"from":"bottom","to":"up","when_all":["knee_ang > 120"]},
              {"from":"up","to":"top","when_all":["knee_ang >= 160"], "rep_complete": true}
            ],
            "errors": [
              {"when_all":["depth < -10"], "message":"Scendi un po’ di più", "penalty":0.3},
              {"when_all":["valgus == 1"], "message":"Ginocchia verso l’esterno", "penalty":0.3}
            ]
          }
        }
      ]
    }
  ]
}
"""

SYSTEM_RUNTIME = """Sei un coach AI in diretta (italiano). Ricevi telemetria e stato del piano.
Devi: motivare, dare correzioni tecniche brevi, gestire tempi/recuperi, e decidere se progredire/regredire o cambiare esercizio/bocco.
Rispondi SOLO JSON con schema:
{
  "coach_action": "say" | "cue" | "progress" | "rest" | "switch_exercise",
  "message": "frase breve in italiano, tono coerente",
  "adjustments": {
    "change_tempo": "string|optional",
    "change_target": {"reps": int|optional, "time_s": int|optional},
    "switch_to": {"block_id":"B2","exercise":"push_up_knees"|null}
  }
}
Mantieni frasi brevi, specifiche, non ripetitive. Se errori si ripetono, proponi regressione intelligente.
"""

def build_plan_prompt(profile:Dict)->str:
    return f"""{SYSTEM_PLAN}
Utente:
{json.dumps({"profile":profile}, ensure_ascii=False)}"""

def build_runtime_prompt(plan_state:Dict, telemetry:Dict, profile:Dict)->str:
    user = {
        "profile": {"level": profile["level"], "goals": profile["goals"], "preferences": profile["preferences"]},
        "plan_state": plan_state,
        "telemetry": telemetry
    }
    return f"""{SYSTEM_RUNTIME}
Dati:
{json.dumps(user, ensure_ascii=False)}
Ricorda: SOLO JSON."""


In [ ]:
# --- Cell 6: Session Orchestrator ---
class SessionOrchestrator:
    def __init__(self, profile:Dict):
        self.profile = profile
        self.pose = PoseEstimator(static_image_mode=False)
        self.plan = None             # piano segreto (JSON LLM)
        self.detectors: Dict[str, DeclarativeDetector] = {}
        self.current_block_idx = 0
        self.current_seq_idx = 0
        self.block_start_ts = None
        self.last_llm_ts = 0.0
        self.llm_period = 2.0        # puoi alzare a 3-5s
        self.last_llm_msg = None
        self.exercise_start_ts = None

    # ----- Plan generation -----
    def generate_plan(self):
        prompt = build_plan_prompt(self.profile)
        plan = ollama_json(prompt)
        self.plan = plan
        self._compile_detectors_from_plan()

        self.current_block_idx = 0
        self.current_seq_idx = 0
        self.block_start_ts = time.time()
        self.exercise_start_ts = time.time()

    def _compile_detectors_from_plan(self):
        self.detectors.clear()
        if not self.plan or "_error" in self.plan: return
        for block in self.plan.get("blocks", []):
            for item in block.get("sequence", []):
                name = item["exercise"]
                tech = item["technique"]
                spec = {
                    "name": name,
                    "initial_phase": tech.get("initial_phase","top"),
                    "vars": tech.get("vars",{}),
                    "transitions": tech.get("transitions",[]),
                    "errors": tech.get("errors",[])
                }
                self.detectors[name] = DeclarativeDetector(spec)

    # ----- Helpers stato corrente -----
    def _current_item(self) -> Optional[Dict]:
        if not self.plan or "_error" in self.plan: return None
        blocks = self.plan.get("blocks", [])
        if self.current_block_idx >= len(blocks): return None
        seq = blocks[self.current_block_idx].get("sequence", [])
        if self.current_seq_idx >= len(seq): return None
        return seq[self.current_seq_idx]

    def _advance_sequence(self):
        item = self._current_item()
        if not item: return
        self.current_seq_idx += 1
        self.exercise_start_ts = time.time()
        # reset detector stato
        ex = item["exercise"]
        if ex in self.detectors: self.detectors[ex].reset()

    def _advance_block(self):
        self.current_block_idx += 1
        self.current_seq_idx = 0
        self.block_start_ts = time.time()

    # ----- Main tick per frame -----
    def process_frame(self, frame_bgr):
        res = self.pose.process(frame_bgr)
        landmarks = res.pose_landmarks if res and res.pose_landmarks else None
        feats = compute_features(landmarks, frame_bgr.shape)

        item = self._current_item()
        if not item:
            out = frame_bgr.copy()
            cv2.putText(out, "Sessione completata. Ottimo lavoro!", (20,40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,180), 2)
            return out, {"done": True, "llm": self.last_llm_msg}

        ex_name = item["exercise"]
        det = self.detectors.get(ex_name)
        rep_event, diag = det.update(landmarks, frame_bgr.shape) if det else (None, {"notes":[]})

        # criteri di avanzamento locali (esempi semplici)
        mode = item.get("mode","reps")
        target = item.get("target",{})
        reps_target = target.get("reps")
        time_target = target.get("time_s")
        rounds = target.get("rounds")
        rest_s = target.get("rest_s", 30)
        now = time.time()
        elapsed_ex = int(now - (self.exercise_start_ts or now))

        # invio telemetria al LLM a cadenza o su rep
        llm_msg = None
        if (now - self.last_llm_ts) > self.llm_period or rep_event is not None:
            telemetry = {
                "time_s": elapsed_ex,
                "exercise": ex_name,
                "rep": det.rep if det else 0,
                "last_rep_quality": getattr(rep_event,"quality", None) if rep_event else None,
                "errors_recent": diag.get("notes",[])[:3],
                "features": {k:v for k,v in feats.items() if isinstance(v,(int,float))}
            }
            plan_state = {
                "block_idx": self.current_block_idx,
                "seq_idx": self.current_seq_idx,
                "item_target": target
            }
            prompt = build_runtime_prompt(plan_state, telemetry, self.profile)
            llm_msg = ollama_json(prompt)
            self.last_llm_ts = now
            if "adjustments" in llm_msg:
                adj = llm_msg["adjustments"] or {}
                if "change_target" in adj and isinstance(adj["change_target"], dict):
                    target.update({k:v for k,v in adj["change_target"].items() if v is not None})
                if "change_tempo" in adj and adj["change_tempo"]:
                    target["tempo"] = adj["change_tempo"]
                if "switch_to" in adj and adj["switch_to"]:
                    # salto diretto se richiesto
                    for bi, block in enumerate(self.plan.get("blocks",[])):
                        if block["id"] == adj["switch_to"].get("block_id"):
                            self.current_block_idx = bi
                            # trova seq con exercise richiesto
                            for si, it in enumerate(block.get("sequence",[])):
                                if it["exercise"] == adj["switch_to"].get("exercise"):
                                    self.current_seq_idx = si
                                    break
                            self.block_start_ts = now
                            self.exercise_start_ts = now
                            break
            self.last_llm_msg = llm_msg

            # se il LLM aggiorna anche le soglie tecniche, ricarica il detector
            # (il messaggio potrebbe includere "technique_update": {...})
            if isinstance(llm_msg, dict) and "technique_update" in llm_msg:
                upd = llm_msg["technique_update"]
                if isinstance(upd, dict) and ex_name in self.detectors:
                    spec = self.detectors[ex_name].spec
                    spec["vars"] = {**spec.get("vars",{}), **upd.get("vars",{})}
                    if "transitions" in upd: spec["transitions"] = upd["transitions"]
                    if "errors" in upd: spec["errors"] = upd["errors"]
                    self.detectors[ex_name].load_spec(spec)

        # avanzamento locale semplice (puoi rifinirlo con piano: serie/round)
        finished = False
        if mode == "reps" and reps_target is not None and det and det.rep >= reps_target:
            finished = True
        if mode == "time" and time_target is not None and elapsed_ex >= time_target:
            finished = True

        if finished:
            # breve fase di "rest" gestita dal LLM
            self._advance_sequence()
            # se il block finisce
            blk = self.plan["blocks"][self.current_block_idx] if self.current_block_idx < len(self.plan["blocks"]) else None
            if blk and self.current_seq_idx >= len(blk.get("sequence",[])):
                self._advance_block()

        # overlay
        out = frame_bgr.copy()
        out = self.pose.draw_landmarks(out, res.pose_landmarks if res else None)
        y=28
        for line in [
            f"Block: {self.current_block_idx+1}/{len(self.plan.get('blocks',[])) if self.plan else 0}",
            f"Exercise: {ex_name}",
            f"Reps: {det.rep if det else 0}",
            f"Elapsed: {elapsed_ex}s"
        ]:
            cv2.putText(out, line, (20,y), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (20,220,20), 2); y+=28

        for c in (diag.get("notes",[])[:2] if diag else []):
            cv2.putText(out, f"• {c}", (20,y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (50,200,255), 2); y+=26

        if isinstance(self.last_llm_msg, dict) and self.last_llm_msg.get("message"):
            cv2.putText(out, self.last_llm_msg["message"], (20,y+10), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255,180,30), 2)

        if DISPLAY_SCALE != 1.0:
            out = cv2.resize(out, None, fx=DISPLAY_SCALE, fy=DISPLAY_SCALE)
        return out, {"diag": diag, "llm": self.last_llm_msg}


In [ ]:
# --- Cell 7: quick tests without camera ---
def test_generate_plan_and_one_image(img_path:str):
    img = cv2.imread(img_path); assert img is not None, f"Immagine non trovata: {img_path}"
    eng = SessionOrchestrator(user_profile)
    eng.generate_plan()  # chiama LLM per creare piano segreto
    frame_out, info = eng.process_frame(img)  # un singolo tick
    cv2.imwrite("debug_output.jpg", frame_out)
    print("LLM last:", info.get("llm"))
    print("Saved overlay to debug_output.jpg")

import glob
def test_folder(folder_glob:str, delay_ms:int=150):
    paths = sorted(glob.glob(folder_glob)); assert paths, f"Nessun file per {folder_glob}"
    eng = SessionOrchestrator(user_profile); eng.generate_plan()
    for p in paths:
        img = cv2.imread(p);
        if img is None: continue
        out, info = eng.process_frame(img)
        cv2.imshow("AI Trainer (sim)", out)
        key = cv2.waitKey(delay_ms) & 0xFF
        if key == 27: break
    cv2.destroyAllWindows()
